In [1]:
import numpy as np
import copy

class Graph:
    def __init__(self):
        self.Nodes = []

    def search(self, name):
        
        exist = False
        
        for node in self.Nodes:
            if(node.name == name):
                exist = True
                break

        if exist:
            return next(node for node in self.Nodes if node.name == name)
        
        else:                       
            new_node = Node(name)
            self.Nodes.append(new_node)
            return new_node

    def addEdge(self, parent, child):
        parent_node = self.search(parent)
        child_node = self.search(child)
        
        if(child_node.name not in parent_node.children):
            parent_node.children.append(child_node)
        
        if(parent_node.name not in child_node.parents):
            child_node.parents.append(parent_node)

    def display(self):
        for node in self.Nodes:
            print(f'{node.name} links to {[child.name for child in node.children]}')

class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = []


In [2]:
def init_graph(fname):
    with open(fname,encoding="utf-8") as f:
        lines = f.readlines()

    graph = Graph()

    for line in lines:
        [parent, child] = line.strip().split(',')
        graph.addEdge(parent, child)
        
    graph.Nodes.sort(key=lambda node: int(node.name))

    return graph


In [3]:
class Similarity:
    def __init__(self, g, decayFactor):
        self.decayFactor = decayFactor
        self.nodeList, self.oldSim = self.init(g)
        self.newSim = [[0] * len(self.nodeList) for i in range(len(self.nodeList))]

    def init(self, g):
        nodeList = [node.name for node in g.Nodes]
        sim = []
        for node1 in nodeList:
            tempSim = []
            for node2 in nodeList:
                if(node1 == node2):
                    tempSim.append(1)
                else:
                    tempSim.append(0)
            sim.append(tempSim)

        return nodeList, sim

    def calSimRank(self, node1, node2):
        if(node1.name == node2.name):
            return 1.0

        pNodes1 = node1.parents
        pNodes2 = node2.parents

        if(len(pNodes1) == 0 or len(pNodes2) == 0):
            return 0.0

        SimRankSum = 0
        for node1 in pNodes1:
            for node2 in pNodes2:
                node1Idx = self.nodeList.index(node1.name)
                node2Idx = self.nodeList.index(node2.name)
                SimRankSum += self.oldSim[node1Idx][node2Idx]

        newSimRank = (self.decayFactor / (len(pNodes1) * len(pNodes2))) * SimRankSum

        return newSimRank

    def get_sim_matrix(self):
        return np.round(np.asarray(self.newSim), 3)

In [4]:
def SimRank(g, sim, num):
    for i in range(num):
        for node1 in g.Nodes:
            for node2 in g.Nodes:
                rank = sim.calSimRank(node1, node2)
                node1_idx = sim.nodeList.index(node1.name)
                node2_idx = sim.nodeList.index(node2.name)
                sim.newSim[node1_idx][node2_idx] = rank

                
        sim.oldSim = copy.deepcopy(sim.newSim)


import os
import time
if __name__ == '__main__':

    decay_factor = 0.8
    it = 30
    result_dir = 'result'
    data_path = './hw3dataset/'
    for file_name in os.listdir(data_path):
        if(file_name == "IBM.txt" or file_name == "graph_6.txt"):
            continue
        file_path = data_path + file_name
        fname = file_path.split('/')[-1].split('.')[0]
        simrank_fname = '_SimRank.txt'

        graph = init_graph(file_path)

        sim = Similarity(graph, decay_factor)
        start = time.time()

        SimRank(graph, sim, it)
        end = time.time()
        
        
        print(fname)
        ans = sim.get_sim_matrix()
        print('SimRank:')
        print(ans)
        print("time: ",end-start)
        print()
        path = os.path.join(result_dir, fname)
        os.makedirs(path, exist_ok=True)
        np.savetxt(os.path.join(path, fname + simrank_fname), ans, delimiter=' ', fmt='%.3f')


graph_4
SimRank:
[[1.    0.36  0.349 0.354 0.338 0.415 0.292]
 [0.36  1.    0.407 0.37  0.412 0.285 0.454]
 [0.349 0.407 1.    0.45  0.39  0.448 0.451]
 [0.354 0.37  0.45  1.    0.343 0.535 0.535]
 [0.338 0.412 0.39  0.343 1.    0.273 0.412]
 [0.415 0.285 0.448 0.535 0.273 1.    0.27 ]
 [0.292 0.454 0.451 0.535 0.412 0.27  1.   ]]
time:  0.010146141052246094

graph_2
SimRank:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
time:  0.0018856525421142578

graph_3
SimRank:
[[1.    0.    0.667 0.   ]
 [0.    1.    0.    0.667]
 [0.667 0.    1.    0.   ]
 [0.    0.667 0.    1.   ]]
time:  0.001184225082397461

graph_1
SimRank:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
time:  0.0010101795196533203

graph_5
SimRank:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]